# Interest Rate Swaps Fundamentals

This notebook demonstrates how to access and use the functionalities of **Swaps** which are part of our **QPS** module within LSEG Financial Analytics SDK.

In [1]:
from lseg_analytics.instruments.ir_swaps import search, load, delete

Search code:

In [2]:
# execute the search of IR swaps
available_swaps = search()

print(available_swaps)

[{'type': 0, 'id': 'c594359d-f1ca-472a-af1a-915b6e742b2f', 'location': {'space': 'HOME', 'name': 'Dummy_OisSwap_EUR'}, 'description': {'summary': '', 'tags': []}}, {'type': 0, 'id': '3aafd38f-9aa7-4111-81ea-171dc62033c2', 'location': {'space': 'HOME', 'name': 'TestFxSp17442139226868882'}, 'description': {'summary': 'Test description', 'tags': ['tag1', 'tag2']}}, {'type': 0, 'id': 'bcd2386c-2402-4681-8190-5b1bf6d5eca7', 'location': {'space': 'HOME', 'name': 'TestIrSwap17500707400624428'}, 'description': {'summary': 'Test ir_swap_saved description', 'tags': ['tag1', 'tag2']}}, {'type': 0, 'id': '4f276496-ae11-4ce0-9870-cc3262fae588', 'location': {'space': 'HOME', 'name': 'TestSwapResource3'}, 'description': {'summary': '(overwritten)', 'tags': ['test']}}, {'type': 0, 'id': 'a09cf5b6-9d05-4aea-b7c4-a167e0cfc6e9', 'location': {'space': 'MYSPACE', 'name': 'TestFxSpotClone17442142162523232'}, 'description': {'summary': 'Test ir_swap_saved description', 'tags': ['tag1', 'tag2']}}, {'type': 0,

### Loading a specific swap template

User can load a specific swap template by either providing a unique template name or resource ID.

Mandatory arguments: 
- resource_id - mutually exclusive with name
- name - mutually exclusive with resource_id

In [3]:
# fetch all available swaps
available_swaps = search()

# execute the load of a swap using the first element of previously fetched data
loaded_swap = load(resource_id=available_swaps[0].id)

print(loaded_swap)

<IrSwap space='HOME' name='Dummy_OisSwap_EUR' c594359d‥>


### Deleting a specific swap template

In [4]:
# fetch all available swaps
available_swaps = search()

# execute the load of a swap using the first element of previously fetched data
loaded_swap = load(resource_id=available_swaps[0].id)

# Clone swap to save original
cloned_swap = loaded_swap.clone()

# Save the cloned swap to a space
cloned_swap.save(name='swap_for_deletion', space='HOME')

# Delete cloned swap
delete(resource_id=cloned_swap.id)


True

## Valuing interest rate swaps

### Imports

Import the following necessary modules:

In [5]:
from lseg_analytics.instruments.ir_swaps import IrSwapDefinitionInstrument, IrPricingParameters, IrSwapSolvingParameters, IrSwapSolvingVariable, IrSwapSolvingTarget, IrMeasure, IrPricingParameters
from lseg_analytics.instruments.ir_swaps import solve, value, create_from_vanilla_irs_template, create_from_cbs_template, create_from_ccs_template, create_from_leg_template, create_from_tbs_template

import json as js

### Template-based setup for swap valuation

Using an existing swap template to build a swap, user can adjust it with his own settings.
- Swap conventions provide a standardized set of market-accepted terms, such as leg frequencies and interest calculation methods.
- By referencing a swap convention, you ensure alignment with market standards while streamlining the setup process.

In [6]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

fwd_start_sofr_def = IrSwapDefinitionInstrument(definition = fwd_start_sofr.definition)

print(js.dumps(fwd_start_sofr_def.as_dict(), indent=4))

{
    "definition": {
        "firstLeg": {
            "rate": {
                "interestRateType": "FixedRate",
                "rate": {
                    "value": 0.0,
                    "unit": "Percentage"
                }
            },
            "interestPeriods": {
                "startDate": {
                    "dateType": "RelativeAdjustableDate",
                    "tenor": "0D",
                    "referenceDate": "SpotDate"
                },
                "endDate": {
                    "dateType": "RelativeAdjustableDate",
                    "tenor": "10Y",
                    "referenceDate": "StartDate"
                },
                "frequency": "Annual",
                "businessDayAdjustment": {
                    "calendars": [
                        "USA"
                    ],
                    "convention": "NextBusinessDay"
                },
                "rollConvention": "Same"
            },
            "paymentOffset": {
        

### Solve a SOFR swap par rate

Solving a swap for a prefered value (e.g. fixed rate or spread) can be done in 2 ways:
1. Invoking a general ir_swaps method 'solve' which then requires as an input argument a swap definition object (IRSwapDefinitionInstrument)
2. Invoking a 'solve' method directly on the IRSwap Object

Both ways require preparation of execution parameters in forms of solving variable and solving target(s), as displayed in following codes.

#### General solve method

In [7]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

# prepare the Definition Instrument
fwd_start_sofr_def = IrSwapDefinitionInstrument(definition = fwd_start_sofr.definition)

# set a solving variable between first and second leg and Fixed Rate or Spread
solving_variable = IrSwapSolvingVariable(leg='FirstLeg', name='FixedRate')

# Apply solving target(s)
solving_target=IrSwapSolvingTarget(market_value=IrMeasure(value=0.0))

# Setup the solving parameter object
solving_parameters = IrSwapSolvingParameters(variable=solving_variable, target=solving_target) 

# instantiate pricing parameters
pricing_parameters = IrPricingParameters(solving_parameters=solving_parameters)

# solve the swap par rate
solving_response_general = solve(
    definitions=[fwd_start_sofr_def],
    pricing_preferences=pricing_parameters
    )

print(js.dumps(solving_response_general.as_dict(), indent=4))

{
    "pricingPreferences": {
        "valuationDate": "2025-08-21",
        "reportCurrency": "USD"
    },
    "analytics": [
        {
            "solving": {
                "result": 3.7728671040693227
            },
            "description": {
                "instrumentTag": "",
                "instrumentDescription": "Pay USD Annual 3.77% vs Receive USD Annual +0bp SOFR 2035-08-27",
                "startDate": "2025-08-25",
                "endDate": "2035-08-27",
                "tenor": "10Y"
            },
            "valuation": {
                "accrued": {
                    "value": 0.0,
                    "percent": 0.0,
                    "dealCurrency": {
                        "value": 0.0,
                        "currency": "USD"
                    },
                    "reportCurrency": {
                        "value": 0.0,
                        "currency": "USD"
                    }
                },
                "marketValue": {
             

#### Class solve method

In this case there are a few rules to abide.
1. Swap needs to be saved before solving
2. Each swap needs a unique name
3. Pricing Parameters needs to be instantiated, even if not populated explicitly

In [8]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

# Swap needs to be saved in order for the solve class method to be executable
fwd_start_sofr.save(name="sofr_fwd_start_swap_exm")

# set a solving variable between first and second leg and Fixed Rate or Spread
solving_variable = IrSwapSolvingVariable(leg='FirstLeg', name='FixedRate')

# Apply solving target(s)
solving_target=IrSwapSolvingTarget(market_value=IrMeasure(value=0.0))

# Setup the solving parameter object
solving_parameters = IrSwapSolvingParameters(variable=solving_variable, target=solving_target) 

# instantiate pricing parameters
pricing_parameters = IrPricingParameters(solving_parameters=solving_parameters)

# solve the swap par rate
solving_response_object = fwd_start_sofr.solve(pricing_preferences=pricing_parameters)

delete(name="sofr_fwd_start_swap_exm")

print(js.dumps(solving_response_object.analytics.as_dict(), indent=4))

{
    "solving": {
        "result": 3.7728670986451025
    },
    "description": {
        "instrumentTag": "",
        "instrumentDescription": "Pay USD Annual 3.77% vs Receive USD Annual +0bp SOFR 2035-08-27",
        "startDate": "2025-08-25",
        "endDate": "2035-08-27",
        "tenor": "10Y"
    },
    "valuation": {
        "accrued": {
            "value": 0.0,
            "percent": 0.0,
            "dealCurrency": {
                "value": 0.0,
                "currency": "USD"
            },
            "reportCurrency": {
                "value": 0.0,
                "currency": "USD"
            }
        },
        "marketValue": {
            "value": 4.65661287307739e-10,
            "dealCurrency": {
                "value": 4.65661287307739e-10,
                "currency": "USD"
            },
            "reportCurrency": {
                "value": 4.65661287307739e-10,
                "currency": "USD"
            }
        },
        "cleanMarketValue": {
   

### Value a swap

Valuing a swap can be done in 2 ways:
1. Invoking a general ir_swaps method 'value' which then requires as an input argument a swap definition object (IRSwapDefinitionInstrument)
2. Invoking a 'solve' method directly on the IRSwap Object

Both ways require preparation of execution parameters in forms of solving variable and solving target(s), as displayed in following codes.

#### General Value method

In [9]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

fwd_start_sofr_def = IrSwapDefinitionInstrument(definition = fwd_start_sofr.definition)

# instantiate pricing parameters
pricing_parameters = IrPricingParameters()

# value the swap
valuation_response = value(
    definitions=[fwd_start_sofr_def],
    pricing_preferences=pricing_parameters
)

print(js.dumps(valuation_response.analytics[0].valuation.as_dict(), indent=4))

{
    "accrued": {
        "value": 0.0,
        "percent": 0.0,
        "dealCurrency": {
            "value": 0.0,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 0.0,
            "currency": "USD"
        }
    },
    "marketValue": {
        "value": 3155746.00060931,
        "dealCurrency": {
            "value": 3155746.00060931,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 3155746.00060931,
            "currency": "USD"
        }
    },
    "cleanMarketValue": {
        "value": 3155746.00060931,
        "dealCurrency": {
            "value": 3155746.00060931,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 3155746.00060931,
            "currency": "USD"
        }
    }
}


#### Class Value method

In [10]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

# Swap needs to be saved in order for the value class method to be executable
fwd_start_sofr.save(name="sofr_fwd_start_swap_exm")

# instantiate pricing parameters
pricing_parameters = IrPricingParameters()

# solve the swap par rate
valuing_response_object = fwd_start_sofr.value(pricing_preferences=pricing_parameters)

delete(name="sofr_fwd_start_swap_exm")

print(js.dumps(valuation_response.analytics[0].valuation.as_dict(), indent=4))

{
    "accrued": {
        "value": 0.0,
        "percent": 0.0,
        "dealCurrency": {
            "value": 0.0,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 0.0,
            "currency": "USD"
        }
    },
    "marketValue": {
        "value": 3155746.00060931,
        "dealCurrency": {
            "value": 3155746.00060931,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 3155746.00060931,
            "currency": "USD"
        }
    },
    "cleanMarketValue": {
        "value": 3155746.00060931,
        "dealCurrency": {
            "value": 3155746.00060931,
            "currency": "USD"
        },
        "reportCurrency": {
            "value": 3155746.00060931,
            "currency": "USD"
        }
    }
}


## Creating a swap from CBS template

In [11]:
swap_from_cbs = create_from_cbs_template(template_reference = "LSEG/GBUSSOSRBS")
print(swap_from_cbs.definition)

{'firstLeg': {'rate': {'interestRateType': 'FloatingRate', 'index': 'LSEG/GBP_SONIA_ON_BOE', 'spreadSchedule': [{'rate': {'value': 0.0, 'unit': 'BasisPoint'}}], 'resetDates': {'offset': {'tenor': '0D', 'businessDayAdjustment': {'calendars': [], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Backward'}}, 'leverage': 1.0}, 'interestPeriods': {'startDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '0D', 'referenceDate': 'SpotDate'}, 'endDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '10Y', 'referenceDate': 'StartDate'}, 'frequency': 'Quarterly', 'businessDayAdjustment': {'calendars': ['UKG', 'USA'], 'convention': 'ModifiedFollowing'}, 'rollConvention': 'Same'}, 'paymentOffset': {'tenor': '2D', 'businessDayAdjustment': {'calendars': ['UKG', 'USA'], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Forward'}, 'couponDayCount': 'Dcb_Actual_365', 'accrualDayCount': 'Dcb_Actual_365', 'principal': {'currency': 

## Create a swap from CCS Template

In [12]:
swap_from_ccs = create_from_ccs_template(template_reference = "LSEG/CNUSQMSRBS")
print(swap_from_ccs.definition)

{'firstLeg': {'rate': {'interestRateType': 'FixedRate', 'rate': {'value': 0.0, 'unit': 'Percentage'}}, 'interestPeriods': {'startDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '0D', 'referenceDate': 'SpotDate'}, 'endDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '10Y', 'referenceDate': 'StartDate'}, 'frequency': 'Quarterly', 'businessDayAdjustment': {'calendars': ['CHN', 'USA'], 'convention': 'ModifiedFollowing'}, 'rollConvention': 'Same'}, 'paymentOffset': {'tenor': '2D', 'businessDayAdjustment': {'calendars': ['CHN', 'USA'], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Forward'}, 'couponDayCount': 'Dcb_Actual_360', 'accrualDayCount': 'Dcb_Actual_360', 'principal': {'currency': 'CNY', 'amount': 10000000.0, 'initialPrincipalExchange': False, 'finalPrincipalExchange': True, 'interimPrincipalExchange': False, 'repaymentCurrency': 'CNY'}, 'payer': 'Party1', 'receiver': 'Party2'}, 'secondLeg': {'rate': {'interestRateType': 'FloatingRat

## Create a swap from LEG Template

In [13]:
swap_from_leg = create_from_leg_template(first_leg_reference = "LSEG/EUR_AB3E_FLT", second_leg_reference = "LSEG/EUR_AB3E_FXD")
print(swap_from_leg.definition)

{'firstLeg': {'rate': {'interestRateType': 'FloatingRate', 'index': 'LSEG/EUR_EURIBOR_3M_EMMI', 'spreadSchedule': [{'rate': {'value': 0.0, 'unit': 'BasisPoint'}}], 'resetDates': {'offset': {'tenor': '2D', 'businessDayAdjustment': {'calendars': [], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodStartDate', 'direction': 'Backward'}}, 'leverage': 1.0}, 'interestPeriods': {'startDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '0D', 'referenceDate': 'SpotDate'}, 'endDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '10Y', 'referenceDate': 'StartDate'}, 'frequency': 'Quarterly', 'businessDayAdjustment': {'calendars': ['EMU'], 'convention': 'ModifiedFollowing'}, 'rollConvention': 'Same'}, 'paymentOffset': {'tenor': '0D', 'businessDayAdjustment': {'calendars': ['EMU'], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Forward'}, 'couponDayCount': 'Dcb_Actual_360', 'accrualDayCount': 'Dcb_Actual_360', 'principal': {'currency': 'EUR', 'a

## Create a swap from TBS Template

In [14]:
swap_from_tbs = create_from_tbs_template(template_reference = "LSEG/CBS_USDSR3LIMM")
print(swap_from_tbs.definition)

{'firstLeg': {'rate': {'interestRateType': 'FloatingRate', 'index': 'LSEG/USD_SOFR_ON', 'spreadSchedule': [{'rate': {'value': 0.0, 'unit': 'BasisPoint'}}], 'resetDates': {'offset': {'tenor': '0D', 'businessDayAdjustment': {'calendars': [], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Backward'}}, 'leverage': 1.0}, 'interestPeriods': {'startDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '0D', 'referenceDate': 'SpotDate'}, 'endDate': {'dateType': 'RelativeAdjustableDate', 'tenor': '10Y', 'referenceDate': 'StartDate'}, 'frequency': 'Quarterly', 'businessDayAdjustment': {'calendars': ['USA'], 'convention': 'ModifiedFollowing'}, 'rollConvention': 'Same'}, 'paymentOffset': {'tenor': '2D', 'businessDayAdjustment': {'calendars': ['USA'], 'convention': 'ModifiedFollowing'}, 'referenceDate': 'PeriodEndDate', 'direction': 'Forward'}, 'couponDayCount': 'Dcb_Actual_360', 'accrualDayCount': 'Dcb_Actual_360', 'principal': {'currency': 'USD', 'amount': 10

### Saving a swap

In [15]:
# build the swap from 'LSEG/OIS_SOFR' template
fwd_start_sofr = create_from_vanilla_irs_template(template_reference = "LSEG/OIS_SOFR")

swap_id = "SOFR_OIS_1Y2Y"

swap_space = "HOME"

try:
    # If the instrument does not exist in HOME space, we can save it
    fwd_start_sofr.save(name=swap_id, space=swap_space)
    print(f"Instrument {swap_id} saved in {swap_space} space.")
except:
    # Check if the instrument already exists in HOME space
    fwd_start_sofr = load(name=swap_id, space=swap_space)
    print(f"Instrument {swap_id} already exists in {swap_space} space.")

Instrument SOFR_OIS_1Y2Y saved in HOME space.


### Deleting the saved swap

In [16]:
# Let's delete the instrument we created in HOME space
from lseg_analytics.instruments.ir_swaps import delete

swap_id = "SOFR_OIS_1Y2Y"

delete(name=swap_id, space="HOME")

True